In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
class BenchmarkResults:
    def __init__(self, filename):
        with open(filename, 'r') as file:
            data = json.load(file)
        self.df = pd.DataFrame(data['benchmarks'])
        self.df[['fixture_name','benchmark_name','n']] = self.df.name.str.split('/', expand=True)
        self.df.n = self.df.n.astype(int)
    def pivot(self):
        return self.df.pivot(index='n', columns='benchmark_name', values='cpu_time')
    def plot(self, *ref_complexity):
        n = self.df.n.unique()
        n_min, n_max = min(n), max(n)
        # Compute the geometric mean of the benchmarks
        y_geomean = np.exp(np.log(
            self.df.groupby("benchmark_name").cpu_time.first()
        ).mean())
        f, ax = plt.subplots(figsize=(8,4))
        ax.set(xscale="log", yscale="log")
        ax.set(xticks=n)
        ax.set(xticklabels=n)
        ax.minorticks_off()
        sns.lineplot(ax=ax, data=self.df, x="n", y="cpu_time", hue="benchmark_name", marker='o')
        for exponent in ref_complexity:
            ax.plot([n_min, n_max], [y_geomean, y_geomean * (n_max/n_min)**exponent], label=f"O(N^{exponent})", linestyle='--')
        handles, labels = ax.get_legend_handles_labels()
        ax.legend(handles, labels)

In [ ]:
(bGemm := BenchmarkResults('bGemm.json')).df.head(4)

In [ ]:
bGemm.pivot()

In [ ]:
bGemm.plot(2, 3)

In [ ]:
(bGemmSweep := BenchmarkResults('bGemmSweep.json')).pivot()

In [ ]:
bGemmSweep.plot(-2)